In [1]:
import os
import numpy as np
from pathlib import Path
import torch
from torch import nn


In [2]:
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

bindata = {}
for fp in sorted(Path("data/ManipNetBIN").iterdir()):
    # print(fp)
    bindata[fp.stem] = np.fromfile(fp, dtype=np.float32)
for k, v in bindata.items():
    print(k, v.shape)

os.chdir("data/ManipNetBIN")
print(os.getcwd())
# num = np.fromfile('Decoder_b0.bin')
# print(num)

/home/shugo/workspace/rendering/network
/home/shugo/workspace/rendering
Decoder_b0 (214,)
Decoder_w0 (328704,)
DenseRes0_b0 (1536,)
DenseRes0_w0 (2359296,)
DenseRes1_b0 (1536,)
DenseRes1_w0 (2359296,)
DenseRes2_b0 (1536,)
DenseRes2_w0 (2359296,)
DenseRes3_b0 (1536,)
DenseRes3_w0 (2359296,)
Encoder0_b0 (512,)
Encoder0_w0 (98304,)
Encoder1_b0 (512,)
Encoder1_w0 (172032,)
Encoder2_b0 (512,)
Encoder2_w0 (932864,)
Xmean (2350,)
Xstd (2350,)
Ymean (214,)
Ystd (214,)
/home/shugo/workspace/rendering/data/ManipNetBIN


In [3]:
import torch.nn.functional as F

class ResBlock(nn.Module):
    def __init__(self, in_vec, out_vec):
        super(ResBlock, self).__init__()
        self.FC0= nn.Linear(in_vec, out_vec)
        self.FC1 = nn.Linear(in_vec, out_vec)
        # self.relu = nn.ReLU()

        # self.sequence = nn.Sequential(
        #     [
        #         nn.Linear(),
        #         nn.ReLU(),
        #     ]
        # )

    def __call__(self, H_zero):
        # H_zero = nn.ReLU(H_zero)
        H_zero_ = F.relu(H_zero)
        H_one = self.FC0(H_zero_) + H_zero_
        # H_one  = nn.ReLU(H_one)
        H_one_ = F.relu(H_one)
        H_two = self.FC1(H_one_) + H_one_ + H_zero_
        return H_two


In [4]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.Encoder0= nn.Linear(192, 512)
        self.Encoder1 = nn.Linear(336, 512)
        self.Encoder2 = nn.Linear(1822, 512)
        self.DenseRes0 = ResBlock(1536, 1536)
        self.DenseRes1 = ResBlock(1536, 1536)
        self.Decoder = nn.Linear(1536, 214)

    def forward(self, pose, traj, sensor):
        pose_ = self.Encoder0(pose)
        traj_ = self.Encoder1(traj)
        sensor_ = self.Encoder2(sensor)
        out = torch.cat([pose_, traj_, sensor_], dim=-1)
        out = self.DenseRes0(out)
        out = self.DenseRes1(out)
        out = self.Decoder(out)
        return out

model = NeuralNetwork().to(device)
print(device)
print(model)
# model()
# print(model.device)
# for param in model.Encoder1.parameters():
#     print("encoder", param.shape)
pose, traj, sensor = torch.zeros([1,192]).to(device), torch.zeros([1,336]).to(device), torch.zeros([1,1822]).to(device)
y = model(pose, traj, sensor)

Using cuda:0 device
cuda:0
NeuralNetwork(
  (Encoder0): Linear(in_features=192, out_features=512, bias=True)
  (Encoder1): Linear(in_features=336, out_features=512, bias=True)
  (Encoder2): Linear(in_features=1822, out_features=512, bias=True)
  (DenseRes0): ResBlock(
    (FC0): Linear(in_features=1536, out_features=1536, bias=True)
    (FC1): Linear(in_features=1536, out_features=1536, bias=True)
  )
  (DenseRes1): ResBlock(
    (FC0): Linear(in_features=1536, out_features=1536, bias=True)
    (FC1): Linear(in_features=1536, out_features=1536, bias=True)
  )
  (Decoder): Linear(in_features=1536, out_features=214, bias=True)
)


In [14]:
for name, param in model.named_parameters():
    print(name, param.dtype)
    # print(param)

Encoder0.weight torch.float32
Encoder0.bias torch.float32
Encoder1.weight torch.float32
Encoder1.bias torch.float32
Encoder2.weight torch.float32
Encoder2.bias torch.float32
DenseRes0.FC0.weight torch.float32
DenseRes0.FC0.bias torch.float32
DenseRes0.FC1.weight torch.float32
DenseRes0.FC1.bias torch.float32
DenseRes1.FC0.weight torch.float32
DenseRes1.FC0.bias torch.float32
DenseRes1.FC1.weight torch.float32
DenseRes1.FC1.bias torch.float32
Decoder.weight torch.float32
Decoder.bias torch.float32


In [5]:
for name, param in model.named_parameters():
    print(name, param.shape)

Encoder0.weight torch.Size([512, 192])
Encoder0.bias torch.Size([512])
Encoder1.weight torch.Size([512, 336])
Encoder1.bias torch.Size([512])
Encoder2.weight torch.Size([512, 1822])
Encoder2.bias torch.Size([512])
DenseRes0.FC0.weight torch.Size([1536, 1536])
DenseRes0.FC0.bias torch.Size([1536])
DenseRes0.FC1.weight torch.Size([1536, 1536])
DenseRes0.FC1.bias torch.Size([1536])
DenseRes1.FC0.weight torch.Size([1536, 1536])
DenseRes1.FC0.bias torch.Size([1536])
DenseRes1.FC1.weight torch.Size([1536, 1536])
DenseRes1.FC1.bias torch.Size([1536])
Decoder.weight torch.Size([214, 1536])
Decoder.bias torch.Size([214])


In [6]:
for k, v in bindata.items():
    print(k, v.shape)

Decoder_b0 (214,)
Decoder_w0 (328704,)
DenseRes0_b0 (1536,)
DenseRes0_w0 (2359296,)
DenseRes1_b0 (1536,)
DenseRes1_w0 (2359296,)
DenseRes2_b0 (1536,)
DenseRes2_w0 (2359296,)
DenseRes3_b0 (1536,)
DenseRes3_w0 (2359296,)
Encoder0_b0 (512,)
Encoder0_w0 (98304,)
Encoder1_b0 (512,)
Encoder1_w0 (172032,)
Encoder2_b0 (512,)
Encoder2_w0 (932864,)
Xmean (2350,)
Xstd (2350,)
Ymean (214,)
Ystd (214,)


In [7]:
netlist = ["Decoder"]
net_name_list = []
# for i in range(4):
#     netlist.append("DenseRes" + str(i))
for i in range(3):
    netlist.append("Encoder" + str(i))
print(netlist)
for name in netlist:
    net_name_list.append([name + ".weight", name + "_w0"])
    net_name_list.append([name + ".bias", name + "_b0"])

shape_list = [[214, 1536], [214], [512, 192], [512], [512, 336], [512], [512, 1822], [512]]
for i in range(len(net_name_list)):
    net_name_list[i].append(shape_list[i])
print(net_name_list)

dense_weight_list = []
dense_bias_list = []
for i in range(2):
    for j in range(2):
        dense_weight_list.append(["DenseRes" + str(i) + ".FC" + str(j) + ".weight", "DenseRes" + str(2 * i + j) + "_w0"])
        dense_bias_list.append(["DenseRes" + str(i) + ".FC" + str(j) + ".bias", "DenseRes" + str(2 * i + j) + "_b0"])


['Decoder', 'Encoder0', 'Encoder1', 'Encoder2']
[['Decoder.weight', 'Decoder_w0', [214, 1536]], ['Decoder.bias', 'Decoder_b0', [214]], ['Encoder0.weight', 'Encoder0_w0', [512, 192]], ['Encoder0.bias', 'Encoder0_b0', [512]], ['Encoder1.weight', 'Encoder1_w0', [512, 336]], ['Encoder1.bias', 'Encoder1_b0', [512]], ['Encoder2.weight', 'Encoder2_w0', [512, 1822]], ['Encoder2.bias', 'Encoder2_b0', [512]]]


# 読み込み

In [8]:
getbinname = {
    "Decoder.weight": "Decoder_w0",
    "Decoder.bias": "Decoder_b0",
    "Encoder0.weight": "Encoder0_w0",
    "Encoder0.bias": "Encoder0_b0",
    "Encoder1.weight": "Encoder1_w0",
    "Encoder1.bias": "Encoder1_b0",
    "Encoder2.weight": "Encoder2_w0",
    "Encoder2.bias": "Encoder2_b0",
    "DenseRes0.FC0.weight": "DenseRes0_w0",
    "DenseRes0.FC0.bias": "DenseRes0_b0",
    "DenseRes0.FC1.weight": "DenseRes1_w0",
    "DenseRes0.FC1.bias": "DenseRes1_b0",
    "DenseRes1.FC0.weight": "DenseRes2_w0",
    "DenseRes1.FC0.bias": "DenseRes2_b0",
    "DenseRes1.FC1.weight": "DenseRes3_w0",
    "DenseRes1.FC1.bias": "DenseRes3_b0",

}
# print(os.getcwd())
#
with torch.no_grad():
    for name, param in model.named_parameters():
        print(name, getbinname[name])
        # fp = f"data/ManipNetBIN/{getbinname[name]}.bin"
        fp = f"{getbinname[name]}.bin"
        savednp = np.fromfile(fp, dtype=np.float32)
        print(savednp.shape)
        # savedweight = torch.from_numpy(savednp)
        shape_ = param.shape
        print(shape_)

        # print(shape_, shape_[0], shape_[1])
        param.copy_(torch.from_numpy(savednp.reshape(shape_, order="C"))) 
        # param.copy_(torch.from_numpy(savednp.reshape(shape_, order="F"))) 
        print(param)
        print(savednp)


Encoder0.weight Encoder0_w0
(98304,)
torch.Size([512, 192])
Parameter containing:
tensor([[-0.0751, -0.0893,  0.0815,  ..., -0.0590, -0.0180, -0.0058],
        [-0.0455,  0.0524, -0.0306,  ..., -0.1515,  0.0012, -0.0810],
        [-0.0329,  0.0703, -0.0476,  ...,  0.0215,  0.0864, -0.0461],
        ...,
        [ 0.0060,  0.0643, -0.0282,  ...,  0.0144,  0.0069, -0.0084],
        [-0.0182,  0.0615,  0.0271,  ...,  0.0547,  0.0410, -0.0219],
        [-0.0358,  0.0303,  0.0649,  ..., -0.0151, -0.1224, -0.0107]],
       device='cuda:0', requires_grad=True)
[-0.07508604 -0.08928403  0.08154082 ... -0.01513198 -0.12243388
 -0.01070601]
Encoder0.bias Encoder0_b0
(512,)
torch.Size([512])
Parameter containing:
tensor([-4.3906e-01,  4.7862e-02, -1.1680e+00, -1.3886e+00, -5.3770e-01,
        -3.2809e-01,  1.0509e+00, -4.4401e-01, -7.2452e-01, -3.2332e-01,
        -9.3679e-01, -7.6667e-01, -7.8083e-01, -1.1598e+00, -9.9273e-01,
        -1.1866e+00, -4.9976e-01, -1.4072e-01, -6.7747e-01, -6.7173e-

In [9]:
with torch.no_grad():
    for name, param in model.named_parameters():
        print(name)
        for k, v in bindata.items():
            for i in range(len(net_name_list)):
                # print(net_name_list[i][0], name)
                if name == net_name_list[i][0] and k == net_name_list[i][1]:
                    print(param.shape, name, k)
                    rephape_weight = torch.reshape(torch.from_numpy(bindata[net_name_list[i][1]]), net_name_list[i][2])
                    param.copy_(rephape_weight)
        for i in range(len(dense_weight_list)):
            if name == dense_weight_list[i][0]:
                param.copy_(torch.reshape(torch.from_numpy(bindata[dense_weight_list[i][1]]), [1536, 1536]))
            if name == dense_bias_list[i][0]:
                param.copy_(torch.reshape(torch.from_numpy(bindata[dense_bias_list[i][1]]), [1536]))


Encoder0.weight
torch.Size([512, 192]) Encoder0.weight Encoder0_w0
Encoder0.bias
torch.Size([512]) Encoder0.bias Encoder0_b0
Encoder1.weight
torch.Size([512, 336]) Encoder1.weight Encoder1_w0
Encoder1.bias
torch.Size([512]) Encoder1.bias Encoder1_b0
Encoder2.weight
torch.Size([512, 1822]) Encoder2.weight Encoder2_w0
Encoder2.bias
torch.Size([512]) Encoder2.bias Encoder2_b0
DenseRes0.FC0.weight
DenseRes0.FC0.bias
DenseRes0.FC1.weight
DenseRes0.FC1.bias
DenseRes1.FC0.weight
DenseRes1.FC0.bias
DenseRes1.FC1.weight
DenseRes1.FC1.bias
Decoder.weight
torch.Size([214, 1536]) Decoder.weight Decoder_w0
Decoder.bias
torch.Size([214]) Decoder.bias Decoder_b0


In [10]:
with torch.no_grad():
    for name, param in model.named_parameters():
        print(name, param)

Encoder0.weight Parameter containing:
tensor([[-0.0751, -0.0893,  0.0815,  ..., -0.0590, -0.0180, -0.0058],
        [-0.0455,  0.0524, -0.0306,  ..., -0.1515,  0.0012, -0.0810],
        [-0.0329,  0.0703, -0.0476,  ...,  0.0215,  0.0864, -0.0461],
        ...,
        [ 0.0060,  0.0643, -0.0282,  ...,  0.0144,  0.0069, -0.0084],
        [-0.0182,  0.0615,  0.0271,  ...,  0.0547,  0.0410, -0.0219],
        [-0.0358,  0.0303,  0.0649,  ..., -0.0151, -0.1224, -0.0107]],
       device='cuda:0', requires_grad=True)
Encoder0.bias Parameter containing:
tensor([-4.3906e-01,  4.7862e-02, -1.1680e+00, -1.3886e+00, -5.3770e-01,
        -3.2809e-01,  1.0509e+00, -4.4401e-01, -7.2452e-01, -3.2332e-01,
        -9.3679e-01, -7.6667e-01, -7.8083e-01, -1.1598e+00, -9.9273e-01,
        -1.1866e+00, -4.9976e-01, -1.4072e-01, -6.7747e-01, -6.7173e-01,
        -1.0881e+00, -8.0475e-02, -6.2963e-01, -1.3231e+00, -6.4958e-01,
        -1.0807e+00, -1.9882e-01, -9.7029e-01, -6.6112e-01, -9.4134e-01,
        -3

In [11]:
# test data
# model.eval()
# 2350 = 192 + 336 + 1822

# x, y = torch.from_numpy(bindata["Xstd"]), torch.from_numpy(bindata["Ystd"])
# with torch.no_grad():
#     pred = model.forward(x[:192], x[192:528], x[528:])
#     # predicted, actual = classes[pred[0].argmax(0)], classes[y]
#     print(f'Predicted: {pred}')
#     print(f"Actual: {y}")
#     print()

In [12]:
Xmean = np.fromfile("Xmean.bin", dtype=np.float32)
Ymean = np.fromfile("Ymean.bin", dtype=np.float32)
print(Xmean.shape, Xmean[-20:])
print(Ymean.shape, Ymean[-20:])

(2350,) [-0.60543  0.40897  0.19042 -0.65405  0.28228  0.40573  0.32636 -0.09081
 -0.31171 -0.26593  0.21367 -0.28124 -0.47571  0.11879 -0.12313 -0.57528
  0.086    0.15045 -0.63428  0.14261]
(214,) [0.95537 0.79754 0.50043 0.7199  0.58381 0.49715 0.80825 0.69686 0.54392
 0.85541 0.74866 0.62923 0.92144 0.81804 0.71969 0.58138 0.61554 0.73037
 0.74607 0.75013]


In [13]:
# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.MSELoss()
model.to(device)

def test(bindata, model):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        x, y = torch.from_numpy(bindata["Xmean"]), torch.from_numpy(bindata["Ymean"])
        pose = x[:192].to(device).unsqueeze(0)
        traj = x[192:528].to(device).unsqueeze(0)
        sensor = x[528:].to(device).unsqueeze(0)
        y = y.to(device).unsqueeze(0)
        pred = model(pose, traj, sensor)
        test_loss = loss_fn(pred, y)
    
    torch.set_printoptions(threshold=10_000)
    print(f"Input: {x[:16]}")
    print(f"Actual: {y.squeeze(0)[:16]}")
    print(f'Predicted: {pred.squeeze(0)[:16]}')
    print(f"Loss: {test_loss}")

test(bindata, model)

Input: tensor([ 0.0000,  0.0000,  0.0000, -0.2251,  0.1177, -0.0623, -0.4225,  0.2583,
        -0.1927, -0.7196,  0.3673, -0.3482, -1.0511,  0.4581, -0.4679, -1.0769])
Actual: tensor([ 0.0000,  0.0000,  0.0000, -0.2251,  0.1177, -0.0623, -0.4225,  0.2583,
        -0.1927, -0.7196,  0.3673, -0.3482, -1.0511,  0.4581, -0.4679, -1.0769],
       device='cuda:0')
Predicted: tensor([-0.0098,  0.0049,  0.0058, -0.0034, -0.0021, -0.0215, -0.2200,  0.0436,
        -0.1534, -0.4226,  0.0810, -0.0447, -0.4017,  0.2072,  0.0113,  0.0012],
       device='cuda:0')
Loss: 0.20533616840839386
